In [1]:
import os
import numpy as np
import pandas as pd
from scipy.spatial import ConvexHull
from itertools import combinations
from progress.bar import Bar

import postures

# Create all features
# Training data
f = 'D_train.csv'
# path  = os.path.dirname(__file__)
# f_path = os.path.relpath('../data/' + f, path)
# f_path = os.path.relpath('../data/' + f)
# df = pd.read_csv(f_path)
df = pd.read_csv('data/D_train.csv')
print('Processing ' + f)
df_new = postures.create_features(df)
df_new.to_csv('../data/D_train_features_all.csv', index = False)

Processing D_train.csv


KeyboardInterrupt: 